# SparkSQL

https://spark.apache.org/docs/latest/sql-ref.html

In [1]:
# Installing required packages
!pip install pyspark
!pip install findspark
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=70f785efc10778299413a3853febdbbdf96782a8998dca333549d10034a3f209
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


## Imports & Load Data

In [2]:
import pandas as pd
import numpy as np

from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark import SparkContext

import findspark
findspark.init()

df = pd.read_csv('cleaned_music_streaming.csv')

df.head()

,artist,track,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration,time_signature,genre
0,Bruno Mars,That's What I Like (feat. Gucci Mane),60.0,0.854,0.564,1.0,-4.964,1,0.0485,0.017100,0.177348,0.0849,0.8990,134.071,234.596,4,5
1,Boston,Hitch a Ride,54.0,0.382,0.814,3.0,-7.230,1,0.0406,0.001100,0.004010,0.1010,0.5690,116.454,251.733,4,10
2,The Raincoats,No Side to Fall In,35.0,0.434,0.614,6.0,-8.334,1,0.0525,0.486000,0.000196,0.3940,0.7870,147.681,109.667,4,6
3,Deno,Lingo (feat. J.I & Chunkz),66.0,0.853,0.597,10.0,-6.528,0,0.0555,0.021200,0.177348,0.1220,0.5690,107.033,173.968,4,5
4,Red Hot Chili Peppers,Nobody Weird Like Me - Remastered,53.0,0.167,0.975,2.0,-4.279,1,0.2160,0.000169,0.016100,0.1720,0.0918,199.060,229.960,4,10


## Create Spark Context and Sessoin

In [3]:
# Context
SparkContext = SparkContext()

# Session
spark = SparkSession \
    .builder \
    .appName("Python Spark DataFrames basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [4]:
spark_df = spark.createDataFrame(df)

## PrintSchema & Show

In [5]:
spark_df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- track: string (nullable = true)
 |-- popularity: double (nullable = true)
 |-- danceability: double (nullable = true)
 |-- energy: double (nullable = true)
 |-- key: double (nullable = true)
 |-- loudness: double (nullable = true)
 |-- mode: long (nullable = true)
 |-- speechiness: double (nullable = true)
 |-- acousticness: double (nullable = true)
 |-- instrumentalness: double (nullable = true)
 |-- liveness: double (nullable = true)
 |-- valence: double (nullable = true)
 |-- tempo: double (nullable = true)
 |-- duration: double (nullable = true)
 |-- time_signature: long (nullable = true)
 |-- genre: long (nullable = true)



In [6]:
spark_df.show(5)

+--------------------+--------------------+----------+------------+------+----+--------+----+-----------+------------+------------------+--------+-------+-------+--------+--------------+-----+
|              artist|               track|popularity|danceability|energy| key|loudness|mode|speechiness|acousticness|  instrumentalness|liveness|valence|  tempo|duration|time_signature|genre|
+--------------------+--------------------+----------+------------+------+----+--------+----+-----------+------------+------------------+--------+-------+-------+--------+--------------+-----+
|          Bruno Mars|That's What I Lik...|      60.0|       0.854| 0.564| 1.0|  -4.964|   1|     0.0485|      0.0171|0.1773476204719195|  0.0849|  0.899|134.071| 234.596|             4|    5|
|              Boston|        Hitch a Ride|      54.0|       0.382| 0.814| 3.0|   -7.23|   1|     0.0406|      0.0011|           0.00401|   0.101|  0.569|116.454| 251.733|             4|   10|
|       The Raincoats|  No Side to 

## Temp View of Data

In [7]:
spark_df.createOrReplaceTempView("music_streaming")

## SparkSQL Operations

### Genre with the highest average popularity

In [8]:
highest_avg_popularity_genre = spark.sql("""
SELECT genre, AVG(popularity) AS avg_popularity
FROM music_streaming
GROUP BY genre
ORDER BY avg_popularity DESC
""")

print(f"Genre with the highest average popularity:")
highest_avg_popularity_genre.show()


Genre with the highest average popularity:
+-----+------------------+
|genre|    avg_popularity|
+-----+------------------+
|    4| 56.77410851584384|
|    9| 54.87524150977302|
|    5| 51.07870208095271|
|   10| 48.40947614489272|
|    1| 45.86965588796474|
|    7| 42.46446578913127|
|    8| 42.30685937906971|
|    6|42.196771683566084|
|    0| 38.97471450826767|
|    2| 33.12738634007679|
|    3|26.772427638735238|
+-----+------------------+



### Artists have recorded the most number of songs with a duration of more than 5 minutes

In [9]:
most_songs_above_5min_artist = spark.sql("""
SELECT artist, COUNT(*) AS num_songs
FROM music_streaming
WHERE duration > 300
GROUP BY artist
ORDER BY num_songs DESC
""")

print("Artists with the most songs above 5 minutes:")
most_songs_above_5min_artist.show()


Artists with the most songs above 5 minutes:
+--------------------+---------+
|              artist|num_songs|
+--------------------+---------+
|           Metallica|       21|
|                TOOL|       13|
|         Arcade Fire|       12|
|        Led Zeppelin|       11|
|           Pearl Jam|        9|
|           Aerosmith|        9|
|               Kyuss|        6|
|Kenny Wayne Shepherd|        6|
|       Wooden Shjips|        6|
|         Sonic Youth|        6|
|             Pantera|        6|
|       Joe Bonamassa|        6|
|               Opeth|        5|
|         Patti Smith|        5|
|        Dire Straits|        5|
|   Avenged Sevenfold|        5|
|      Monster Magnet|        5|
|     Bernard Allison|        5|
|                  U2|        5|
| Derek & The Dominos|        5|
+--------------------+---------+
only showing top 20 rows



### Number of songs are included in every Genre

In [10]:
num_songs_in_each_genre = spark.sql("""
SELECT genre, COUNT(*) AS num_songs
FROM music_streaming
GROUP BY genre
""")

print("Number of songs in each genre:")
num_songs_in_each_genre.show()


Number of songs in each genre:
+-----+---------+
|genre|num_songs|
+-----+---------+
|    0|      586|
|    7|      465|
|    6|     2263|
|    9|     1828|
|    5|     1210|
|    1|     1268|
|   10|     4264|
|    3|      371|
|    8|     1704|
|    2|     1182|
|    4|      376|
+-----+---------+



### Artists dominated the charts

In [11]:
most_popular_artists = spark.sql("""
SELECT artist, SUM(popularity) AS total_popularity
FROM music_streaming
GROUP BY artist
ORDER BY total_popularity DESC
""")

print("Most popular artists:")
most_popular_artists.show()


Most popular artists:
+--------------------+------------------+
|              artist|  total_popularity|
+--------------------+------------------+
|      Britney Spears| 2637.241221979766|
|     Backstreet Boys|            2615.0|
|  The Rolling Stones|1838.3294319910071|
|           Metallica|            1710.0|
|                  U2|            1648.0|
|            Westlife|1553.7236659392977|
|               AC/DC|            1549.0|
|            Coldplay|            1441.0|
|         The Beatles|1409.6588639820143|
|             Nirvana|            1385.0|
|        Led Zeppelin|            1385.0|
|      The Black Keys|            1299.0|
|       Fleetwood Mac|            1248.0|
|Creedence Clearwa...|1181.7470739932553|
|      Arctic Monkeys|            1144.0|
|           Aerosmith|1110.5823579977518|
|           Pearl Jam|            1096.0|
|           Van Halen|            1082.0|
|The Smashing Pump...|1038.5823579977518|
|         The Killers|  992.911789988759|
+-----------

### Least 5 fun/not-boring songs that can be played at a party

In [12]:
party_songs = spark.sql("""
SELECT track AS title, artist, energy, danceability
FROM music_streaming
ORDER BY energy DESC, danceability DESC
LIMIT 5
""")

print("Fun songs for a party:")
party_songs.show()


Fun songs for a party:
+--------------------+----------+------+------------+
|               title|    artist|energy|danceability|
+--------------------+----------+------+------------+
|Transilvanian Hunger|Darkthrone|   1.0|       0.267|
|Never Let You Dow...|     Kayzo| 0.999|       0.446|
|In the Shadow of ...|Darkthrone| 0.999|       0.141|
|           BITE BACK| FEVER 333| 0.998|       0.545|
|        Come with Me|   Stepson| 0.998|       0.468|
+--------------------+----------+------+------------+



# Spark ML

### Split the data into training and testing sets & Vector assembler

Vector assembler : A feature transformer that merges multiple columns into a vector column.

In [13]:
from pyspark.ml.feature import VectorAssembler

train_df, test_df = spark_df.randomSplit([0.8, 0.2])


assembler = VectorAssembler(inputCols=['acousticness',
                                       'danceability',
                                       'energy',
                                       'instrumentalness',
                                       'liveness',
                                       'loudness',
                                       'speechiness',
                                       'tempo',
                                       'valence'], outputCol='features')


### Transform, Train and fit the data in the models

In [14]:
train_df = assembler.transform(train_df)
test_df = assembler.transform(test_df)

lr = LinearRegression(featuresCol='features', labelCol='genre')
dt = DecisionTreeClassifier(featuresCol='features', labelCol='genre')
rf = RandomForestClassifier(featuresCol='features', labelCol='genre')

lr_model = lr.fit(train_df)
dt_model = dt.fit(train_df)
rf_model = rf.fit(train_df)


### Predict and Evaluate the models

In [15]:
from pyspark.ml.evaluation import RegressionEvaluator

lr_predictions = lr_model.transform(test_df)
dt_predictions = dt_model.transform(test_df)
rf_predictions = rf_model.transform(test_df)

evaluator = RegressionEvaluator(labelCol='genre', predictionCol='prediction', metricName='rmse')

lr_rmse = evaluator.evaluate(lr_predictions)
dt_rmse = evaluator.evaluate(dt_predictions)
rf_rmse = evaluator.evaluate(rf_predictions)

print("Linear Regression RMSE:", lr_rmse)
print("Decision Tree RMSE:", dt_rmse)
print("Random Forest RMSE:", rf_rmse)


Linear Regression RMSE: 3.0968818193754775
Decision Tree RMSE: 4.15241155368313
Random Forest RMSE: 4.190387075254883


Best model Linear Regression as it has the lowest RMSE

